In [1]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,event_list_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [35]:
prince_edward_bos(county_dictionary_single_variable['Prince Edward County Board of Supervisors'][1])

[]

In [340]:
"Prince Edward County Board of Supervisors"
def prince_edward_bos(url):
    driver.get(url)
    time.sleep(10)
    messages = []
    #row 0 is the column headers, latest meeting will be index 1
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    future_meetings = [item for item in table_rows[1:] if item.text !="   " and check_meeting_date(item.text.split("Meeting")[1].split("-")[0])==True]
    meeting_links = [item.find_elements(By.CSS_SELECTOR,'a')[1].get_attribute("href") for item in future_meetings]
    for item in meeting_links:
        driver.get(item)
        time.sleep(10)
        pages = driver.find_elements(By.CSS_SELECTOR,"div[class*=page")
        for page in pages:
            driver.execute_script("arguments[0].scrollIntoView();", page)
            time.sleep(2)
            agenda_content = driver.find_elements(By.CSS_SELECTOR,'div[class*=textLayer')
            agenda_search = search_agenda_for_keywords(agenda_content)
            if agenda_search != []:
                messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Prince Edward County Board of Supervisors. " + item)
    return pd.Series(messages).unique().tolist()

[]